In [1]:
# two 96-well plates?
import numpy as np
import pandas as pd

from data_loading import load_annotated_6k_collection

df = pd.read_csv('../output/TF-iso_ref-vs-alt.tsv', sep='\t')
tfs = load_annotated_6k_collection()

reading from cache


In [2]:
ppi_selection = (
    (df['PPI_jaccard'] != 1) & df['PPI_jaccard'].notnull() 
    & (df['n_positive_PPI_ref'] >= 1)
    & (df['n_positive_PPI_alt'] >= 1)
    & (df['n_PPI_successfully_tested_in_ref_and_alt'] >= 3)
                  )
pdi_selection = ((df['PDI_jaccard'] != 1)
                     & df['PDI_jaccard'].notnull()
                     & (df['n_PDI_successfully_tested_in_ref_and_alt'] >= 3))
act_selection = (df['activation_fold_change_log2'].abs() >= np.log2(8))
base_selection = ((ppi_selection | pdi_selection | act_selection)
                  & ~df['is_ref_novel_isoform']
                  & ~df['is_alt_novel_isoform']
                  & df['is_MANE_select_isoform_cloned'])

print(df.loc[base_selection, ['clone_acc_ref', 'clone_acc_alt']].nunique().sum(),
'clones with base selection') 

110 clones with base selection


In [3]:
(ppi_selection.sum(), pdi_selection.sum(), act_selection.sum())

(62, 41, 33)

In [4]:
selection = base_selection
selected_clones = set(df.loc[df['gene_symbol'].isin(
            df.loc[selection, 'gene_symbol'].unique()),
        ['clone_acc_ref', 'clone_acc_alt']].values.flatten())
print(len(selected_clones))

160


In [5]:
hand_picked = {'GRHL3',
               'HEY1',
               'PBX1',
               'NFYA',
               'THRA',
               'THRB',
               'ZIC3',
               'TBX5',
               'TBPL1',
               'RFX3',
               'CREB5',
               'CREB1',
               'DLX4',
               'RORC',
               'ATF2',
               'MAX',
               'PPARG',
               'STAT3',

               'TP53',   # has aggregate evidence for an isoform

               'NFIX', # has 2 isoforms with no PPI or PDI changes
               'GLI1', # has 2 isoforms with no PPI changes

               'TP63',  # from Nidhi's list where variant shows condensate
               'TCF4',  # from Nidhi's list where variant shows condensate
               
               'KLF7', 'PBX1', 'DLX1', 'NFYA', 'PKNOX1', 'ATF2', 'HOXA1', 'MLX',
                'HMBOX1', 'GRHL3', 'HEY1', 'RARB', 'TCF4', 'TGIF1', 'DDIT3', 'TBX5', 'LHX9', 'ZBTB18',
               
               }
hand_picked_clones = set(df.loc[df['gene_symbol'].isin(hand_picked),
                                ['clone_acc_ref', 'clone_acc_alt']].values.flatten())
print(', '.join(hand_picked.difference(set(df.loc[selection, 'gene_symbol'].values))))
print(len(selected_clones.union(hand_picked_clones)))

HOXA1, GLI1, CREB5, NFIX, MLX, GRHL3, THRA, STAT3, TP53, PPARG, ZIC3, RARB
196


In [6]:
df.loc[df['clone_acc_alt'].isin(selected_clones.union(hand_picked_clones))
       & df['is_alt_novel_isoform']
       & df['PPI_jaccard'].isnull()
       & df['PDI_jaccard'].isnull(), :]

,gene_symbol,Ensembl_gene_ID,family,is_dimerizing_TF_family,clone_acc_ref,clone_acc_alt,Ensembl_transcript_IDs_ref,Ensembl_transcript_IDs_alt,is_ref_novel_isoform,is_alt_novel_isoform,is_MANE_select_isoform_cloned,n_aa_ref,n_aa_alt,n_exons_ref,n_exons_alt,is_alternative_N_terminal,is_alternative_C_terminal,is_alternative_internal_exon,is_alternative_5_prime_donor,is_alternative_3_prime_acceptor,is_exon_skipping,is_mutually_exclusive_exons,is_intron_retention,dbd_insertion_n_aa,dbd_pct_lost,dbd_affected,aa_seq_pct_id,n_positive_PPI_ref,n_positive_PPI_alt,n_PPI_successfully_tested_in_ref_and_alt,n_positive_PPI_ref_filtered,n_positive_PPI_alt_filtered,n_shared_PPI,PPI_jaccard,dimer_ppi,other_than_dimer_ppi,tf_tf_ppi,tf_cofactor_ppi,tf_signalling_ppi,n_positive_PDI_ref,n_positive_PDI_alt,n_PDI_successfully_tested_in_ref_and_alt,n_positive_PDI_ref_filtered,n_positive_PDI_alt_filtered,n_shared_PDI,PDI_jaccard,at_least_one_isoform_in_gene_abs_activation_gte_2fold,activation_ref,activation_alt,activation_fold_change_log2
27,RORC,ENSG00000143365,Nuclear receptor,True,RORC|3/6|12D07,RORC|1/6|05F11,ENST00000318247,NaN,False,True,True,518,488,11,10,True,True,False,False,False,False,False,False,0.0,0.000000,False,82.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,True,2.472553,NaN,NaN
151,FOXP2,ENSG00000128573,Forkhead,False,FOXP2|2/10|07B01,FOXP2|6/10|06F12,ENST00000350908|ENST00000393494,NaN,False,True,True,715,450,16,10,False,True,True,True,False,False,False,False,0.0,100.000000,True,57.6,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,True,3.640735,0.767606,-2.873129
164,TFEC,ENSG00000105967,bHLH,True,TFEC|1/6|03F07,TFEC|4/6|03H11,ENST00000265440,NaN,False,True,True,347,226,7,6,False,True,False,True,False,False,False,False,0.0,0.000000,False,63.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,6.330644,3.169927,-3.160717
166,TFEC,ENSG00000105967,bHLH,True,TFEC|1/6|03F07,TFEC|6/6|03E11,ENST00000265440,NaN,False,True,True,347,196,7,5,False,True,True,True,True,True,False,False,1.0,3.703704,True,55.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,6.330644,1.201684,-5.128960


In [7]:
df.loc[df['clone_acc_alt'].isin(selected_clones.union(hand_picked_clones))
       & df['PPI_jaccard'].isnull()
       & df['PDI_jaccard'].isnull()
       & df['activation_alt'].isnull(), :]

,gene_symbol,Ensembl_gene_ID,family,is_dimerizing_TF_family,clone_acc_ref,clone_acc_alt,Ensembl_transcript_IDs_ref,Ensembl_transcript_IDs_alt,is_ref_novel_isoform,is_alt_novel_isoform,is_MANE_select_isoform_cloned,n_aa_ref,n_aa_alt,n_exons_ref,n_exons_alt,is_alternative_N_terminal,is_alternative_C_terminal,is_alternative_internal_exon,is_alternative_5_prime_donor,is_alternative_3_prime_acceptor,is_exon_skipping,is_mutually_exclusive_exons,is_intron_retention,dbd_insertion_n_aa,dbd_pct_lost,dbd_affected,aa_seq_pct_id,n_positive_PPI_ref,n_positive_PPI_alt,n_PPI_successfully_tested_in_ref_and_alt,n_positive_PPI_ref_filtered,n_positive_PPI_alt_filtered,n_shared_PPI,PPI_jaccard,dimer_ppi,other_than_dimer_ppi,tf_tf_ppi,tf_cofactor_ppi,tf_signalling_ppi,n_positive_PDI_ref,n_positive_PDI_alt,n_PDI_successfully_tested_in_ref_and_alt,n_positive_PDI_ref_filtered,n_positive_PDI_alt_filtered,n_shared_PDI,PDI_jaccard,at_least_one_isoform_in_gene_abs_activation_gte_2fold,activation_ref,activation_alt,activation_fold_change_log2
27,RORC,ENSG00000143365,Nuclear receptor,True,RORC|3/6|12D07,RORC|1/6|05F11,ENST00000318247,NaN,False,True,True,518,488,11,10,True,True,False,False,False,False,False,False,0.0,0.0,False,82.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,True,2.472553,NaN,NaN
162,TFEC,ENSG00000105967,bHLH,True,TFEC|1/6|03F07,TFEC|2/6|04A02,ENST00000265440,ENST00000320239,False,False,True,347,318,7,6,False,False,True,False,False,True,False,False,0.0,0.0,False,91.6,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,6.330644,NaN,NaN
165,TFEC,ENSG00000105967,bHLH,True,TFEC|1/6|03F07,TFEC|5/6|04A01,ENST00000265440,ENST00000393485,False,False,True,347,197,7,5,False,True,True,True,False,True,False,False,0.0,0.0,False,55.3,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,6.330644,NaN,NaN
274,MAX,ENSG00000125952,bHLH,True,MAX|1/6|08G02,MAX|2/6|07F12,ENST00000358664,ENST00000358402,False,False,True,160,151,5,4,False,False,True,False,False,True,False,False,0.0,0.0,False,94.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,NaN,NaN,NaN,NaN,NaN,True,1.755586,NaN,NaN


In [8]:
hand_picked_clones_to_drop = {'MLX|2/4|01D12', 'RORC|1/6|05F11', 'TFEC|2/6|04A02', 'TFEC|5/6|04A01', }

In [9]:
selected_clones = selected_clones.union(hand_picked_clones).difference(hand_picked_clones_to_drop)

In [10]:
df.head()

,gene_symbol,Ensembl_gene_ID,family,is_dimerizing_TF_family,clone_acc_ref,clone_acc_alt,Ensembl_transcript_IDs_ref,Ensembl_transcript_IDs_alt,is_ref_novel_isoform,is_alt_novel_isoform,is_MANE_select_isoform_cloned,n_aa_ref,n_aa_alt,n_exons_ref,n_exons_alt,is_alternative_N_terminal,is_alternative_C_terminal,is_alternative_internal_exon,is_alternative_5_prime_donor,is_alternative_3_prime_acceptor,is_exon_skipping,is_mutually_exclusive_exons,is_intron_retention,dbd_insertion_n_aa,dbd_pct_lost,dbd_affected,aa_seq_pct_id,n_positive_PPI_ref,n_positive_PPI_alt,n_PPI_successfully_tested_in_ref_and_alt,n_positive_PPI_ref_filtered,n_positive_PPI_alt_filtered,n_shared_PPI,PPI_jaccard,dimer_ppi,other_than_dimer_ppi,tf_tf_ppi,tf_cofactor_ppi,tf_signalling_ppi,n_positive_PDI_ref,n_positive_PDI_alt,n_PDI_successfully_tested_in_ref_and_alt,n_positive_PDI_ref_filtered,n_positive_PDI_alt_filtered,n_shared_PDI,PDI_jaccard,at_least_one_isoform_in_gene_abs_activation_gte_2fold,activation_ref,activation_alt,activation_fold_change_log2
0,ZBTB18,ENSG00000179456,C2H2 ZF,False,ZBTB18|1/2|04E08,ZBTB18|2/2|12E05,ENST00000358704,ENST00000622512,False,False,True,531,522,2,1,True,False,False,False,True,False,False,False,0.0,0.000000,False,98.1,4.0,3.0,4.0,4.0,3.0,3.0,0.75,NaN,loses some,loses some,retains all,retains all,0.0,1.0,1.0,0.0,1.0,0.0,0.000000,True,2.153413,2.656826,0.503414
1,PRRX1,ENSG00000116132,Homeodomain,False,PRRX1|2/3|09F06,PRRX1|1/3|09C07,ENST00000239461,ENST00000367760,False,False,True,245,217,4,4,False,True,False,False,False,False,False,False,0.0,0.000000,False,82.0,0.0,13.0,13.0,0.0,13.0,0.0,0.00,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,True,1.552087,0.679816,-0.872271
2,PRRX1,ENSG00000116132,Homeodomain,False,PRRX1|2/3|09F06,PRRX1|3/3|09H08,ENST00000239461,ENST00000497230,False,False,True,245,200,4,3,False,True,False,True,False,False,False,False,0.0,0.000000,False,81.2,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,13.0,14.0,14.0,13.0,13.0,0.928571,True,1.552087,0.773129,-0.778958
3,POU2F1,ENSG00000143190,Homeodomain; POU,False,POU2F1|2/2|04D01,POU2F1|1/2|03F12,ENST00000541643,ENST00000367862,False,False,False,743,755,15,16,True,False,False,False,True,False,False,False,0.0,0.000000,False,98.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2.509024,NaN,NaN
4,ATF3,ENSG00000162772,bZIP,True,ATF3|1/2|08B04,ATF3|2/2|08D04,ENST00000341491|ENST00000366987,ENST00000366983,False,False,True,181,135,3,3,False,True,False,False,False,False,False,False,19.0,46.774194,True,65.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.979226,-0.381163,-1.360389


In [11]:
print(len(selected_clones), 'cloned isoforms')
if not ((df.loc[df['clone_acc_alt'].isin(selected_clones), 'clone_acc_ref'].nunique() + 
        df.loc[df['clone_acc_alt'].isin(selected_clones), 'clone_acc_alt'].nunique()) 
        == len(selected_clones)):
    raise UserWarning('something wrong')

print(df.loc[df['clone_acc_alt'].isin(selected_clones), 'clone_acc_ref'].nunique(), 'reference')
print(df.loc[df['clone_acc_alt'].isin(selected_clones), 'gene_symbol'].nunique(),
      'different genes')
print(df.loc[df['clone_acc_alt'].isin(selected_clones), 'clone_acc_alt'].nunique(), 'alternative')
print((df['clone_acc_alt'].isin(selected_clones) & 
       (df['PPI_jaccard'] < 1)
       & (df['n_positive_PPI_ref'] >= 1)
       & (df['n_positive_PPI_alt'] >= 1)).sum(),
      'with PPI differences and both isoforms have at least one PPI')
print((df['clone_acc_alt'].isin(selected_clones) 
       & (df['n_positive_PPI_ref'] >= 1)
       & (df['n_positive_PPI_alt'] == 0)).sum(),
      'lose all PPIs')
print((df['clone_acc_alt'].isin(selected_clones) & (df['PDI_jaccard'] < 1)).sum(),
      'with PDI differences')
print((df['clone_acc_alt'].isin(selected_clones) & (df['PDI_jaccard'] < 1) & (df['n_positive_PDI_alt'] >= 1)).sum(),
      'with PDI differences, retain some DNA binding')
print((df['clone_acc_alt'].isin(selected_clones) & (df['n_positive_PDI_ref'] >= 1) & (df['n_positive_PDI_alt'] == 0)).sum(),
      'lose all DNA binding')
for cutoff in [2, 5, 10]:
      print((df['clone_acc_alt'].isin(selected_clones) & (df['activation_fold_change_log2'].abs() >= np.log2(cutoff))).sum(),
            f'with >= {cutoff}-fold activation difference')

192 cloned isoforms
61 reference
61 different genes
131 alternative
45 with PPI differences and both isoforms have at least one PPI
23 lose all PPIs
45 with PDI differences
25 with PDI differences, retain some DNA binding
20 lose all DNA binding
60 with >= 2-fold activation difference
27 with >= 5-fold activation difference
9 with >= 10-fold activation difference


In [12]:
print((df.loc[df['clone_acc_alt'].isin(selected_clones), 'PPI_jaccard'] == 1).sum(),
      'clones with no PPI difference',
      'out of',
       (df['PPI_jaccard'] == 1).sum(),
       'possible'
      )
print((df.loc[df['clone_acc_alt'].isin(selected_clones), 'PDI_jaccard'] == 1).sum(),
      'clones with no PDI difference',
      'out of',
       (df['PDI_jaccard'] == 1).sum(),
       'possible'
      )
print((df.loc[df['clone_acc_alt'].isin(selected_clones),
             'activation_fold_change_log2'].abs() < 1).sum(),
      'clones with < 2-fold activation difference')

12 clones with no PPI difference out of 31 possible
5 clones with no PDI difference out of 9 possible
49 clones with < 2-fold activation difference


In [13]:
# Add in more controls?
df.loc[~df['clone_acc_alt'].isin(selected_clones) & (df['PPI_jaccard'] == 1), :]
# ARNTL2 GLI1 NFIX

,gene_symbol,Ensembl_gene_ID,family,is_dimerizing_TF_family,clone_acc_ref,clone_acc_alt,Ensembl_transcript_IDs_ref,Ensembl_transcript_IDs_alt,is_ref_novel_isoform,is_alt_novel_isoform,is_MANE_select_isoform_cloned,n_aa_ref,n_aa_alt,n_exons_ref,n_exons_alt,is_alternative_N_terminal,is_alternative_C_terminal,is_alternative_internal_exon,is_alternative_5_prime_donor,is_alternative_3_prime_acceptor,is_exon_skipping,is_mutually_exclusive_exons,is_intron_retention,dbd_insertion_n_aa,dbd_pct_lost,dbd_affected,aa_seq_pct_id,n_positive_PPI_ref,n_positive_PPI_alt,n_PPI_successfully_tested_in_ref_and_alt,n_positive_PPI_ref_filtered,n_positive_PPI_alt_filtered,n_shared_PPI,PPI_jaccard,dimer_ppi,other_than_dimer_ppi,tf_tf_ppi,tf_cofactor_ppi,tf_signalling_ppi,n_positive_PDI_ref,n_positive_PDI_alt,n_PDI_successfully_tested_in_ref_and_alt,n_positive_PDI_ref_filtered,n_positive_PDI_alt_filtered,n_shared_PDI,PDI_jaccard,at_least_one_isoform_in_gene_abs_activation_gte_2fold,activation_ref,activation_alt,activation_fold_change_log2
54,NFE2L2,ENSG00000116044,bZIP,True,NFE2L2|1/3|12H08,NFE2L2|2/3|01F05,ENST00000397062,ENST00000397063|ENST00000464747,False,False,True,605,589,5,4,True,False,False,False,True,False,False,False,0.0,0.000000,False,97.4,4.0,4.0,4.0,4.0,4.0,4.0,1.0,retains all,retains all,retains all,retains all,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,8.724385,9.265946,0.541562
86,RBPJ,ENSG00000168214,CSL,False,RBPJ|2/4|07B11,RBPJ|3/4|07D11,ENST00000342320|ENST00000355476,ENST00000345843,False,False,True,486,485,11,11,True,False,False,False,False,False,False,False,0.0,0.000000,False,99.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,retains all,retains all,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,True,3.279541,3.987183,0.707642
115,HSF2,ENSG00000025156,HSF,True,HSF2|1/2|12B08,HSF2|2/2|09A10,ENST00000368455,ENST00000452194,False,False,True,536,518,13,12,False,False,True,False,False,True,False,False,0.0,0.000000,False,96.6,2.0,2.0,2.0,2.0,2.0,2.0,1.0,NaN,retains all,NaN,retains all,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1.058213,1.523062,0.464849
130,TEAD3,ENSG00000007866,TEA,False,TEAD3|2/2|10G04,TEAD3|1/2|10E03,ENST00000338863,ENST00000639578,False,False,NaN,435,435,12,12,False,False,True,False,False,False,True,False,0.0,15.714286,True,98.2,4.0,4.0,4.0,4.0,4.0,4.0,1.0,NaN,retains all,NaN,retains all,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2.764910,3.113102,0.348192
148,ZNF212,ENSG00000170260,C2H2 ZF,False,ZNF212|1/2|12D01,ZNF212|2/2|03E09,ENST00000335870,NaN,False,True,True,495,494,5,5,False,False,False,False,True,False,False,False,0.0,0.000000,False,99.8,14.0,14.0,14.0,14.0,14.0,14.0,1.0,NaN,retains all,retains all,retains all,retains all,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.115226,-0.130140,-0.245366
191,LHX6,ENSG00000106852,Homeodomain,False,LHX6|1/4|03D01,LHX6|4/4|11G11,ENST00000394319,ENST00000373755,False,False,True,392,363,10,9,True,False,False,False,True,False,False,False,0.0,0.000000,False,92.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,retains all,NaN,retains all,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,True,2.838064,2.262180,-0.575883
244,ARNTL2,ENSG00000029153,bHLH,True,ARNTL2|2/5|12C06,ARNTL2|1/5|10H01,ENST00000395901,NaN,False,True,False,599,633,15,16,False,False,True,False,False,False,False,False,34.0,0.000000,False,94.6,2.0,2.0,2.0,2.0,2.0,2.0,1.0,retains all,NaN,retains all,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.363099,NaN,NaN
245,ARNTL2,ENSG00000029153,bHLH,True,ARNTL2|2/5|12C06,ARNTL2|3/5|10B01,ENST00000395901,NaN,False,True,False,599,585,15,15,True,False,True,False,True,False,False,False,34.0,0.000000,False,87.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,retains all,NaN,retains all,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.363099,0.225330,-0.137769
288,NR2F2,ENSG00000185551,Nuclear receptor,True,NR2F2|1/2|09E07,NR2F2|2/2|09C09,ENST00000421109,ENST00000394171|ENST00000453270,False,False,False,281,261,3,2,True,False,False,False,True,False,False,False,NaN,NaN,False,92.5,2.0,2.0,2.0,2.0,2.0,2.0,1.0,NaN,retains all,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.625239,0.884932

In [14]:
df.loc[~df['clone_acc_alt'].isin(selected_clones) & (df['PDI_jaccard'] == 1), :]

,gene_symbol,Ensembl_gene_ID,family,is_dimerizing_TF_family,clone_acc_ref,clone_acc_alt,Ensembl_transcript_IDs_ref,Ensembl_transcript_IDs_alt,is_ref_novel_isoform,is_alt_novel_isoform,is_MANE_select_isoform_cloned,n_aa_ref,n_aa_alt,n_exons_ref,n_exons_alt,is_alternative_N_terminal,is_alternative_C_terminal,is_alternative_internal_exon,is_alternative_5_prime_donor,is_alternative_3_prime_acceptor,is_exon_skipping,is_mutually_exclusive_exons,is_intron_retention,dbd_insertion_n_aa,dbd_pct_lost,dbd_affected,aa_seq_pct_id,n_positive_PPI_ref,n_positive_PPI_alt,n_PPI_successfully_tested_in_ref_and_alt,n_positive_PPI_ref_filtered,n_positive_PPI_alt_filtered,n_shared_PPI,PPI_jaccard,dimer_ppi,other_than_dimer_ppi,tf_tf_ppi,tf_cofactor_ppi,tf_signalling_ppi,n_positive_PDI_ref,n_positive_PDI_alt,n_PDI_successfully_tested_in_ref_and_alt,n_positive_PDI_ref_filtered,n_positive_PDI_alt_filtered,n_shared_PDI,PDI_jaccard,at_least_one_isoform_in_gene_abs_activation_gte_2fold,activation_ref,activation_alt,activation_fold_change_log2
217,ZNF143,ENSG00000166478,C2H2 ZF,False,ZNF143|1/3|09D07,ZNF143|2/3|09G09,ENST00000396604|ENST00000530463,NaN,False,True,False,637,606,15,14,False,False,True,False,False,True,False,False,0.0,0.0,False,94.8,6.0,3.0,7.0,6.0,2.0,1.0,0.142857,NaN,loses some,loses all,loses some,loses all,1.0,1.0,1.0,1.0,1.0,1.0,1.0,True,6.759239,6.953447,0.194208
218,ZNF143,ENSG00000166478,C2H2 ZF,False,ZNF143|1/3|09D07,ZNF143|3/3|09A08,ENST00000396604|ENST00000530463,NaN,False,True,False,637,584,15,14,False,False,True,False,True,True,False,False,0.0,0.0,False,91.2,6.0,10.0,9.0,6.0,9.0,6.0,0.666667,NaN,gains some,retains all,gains some,retains all,1.0,1.0,1.0,1.0,1.0,1.0,1.0,True,6.759239,6.458807,-0.300432
338,ZNF18,ENSG00000154957,C2H2 ZF,False,ZNF18|2/3|07H02,ZNF18|1/3|07G04,ENST00000322748|ENST00000580306|ENST00000580613,NaN,False,True,True,549,568,6,6,False,False,False,True,True,False,False,False,0.0,0.0,False,96.3,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,NaN,retains all,retains all,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,True,2.982778,2.142965,-0.839813
339,ZNF18,ENSG00000154957,C2H2 ZF,False,ZNF18|2/3|07H02,ZNF18|3/3|07D01,ENST00000322748|ENST00000580306|ENST00000580613,ENST00000454073,False,False,True,549,548,6,6,False,False,False,False,True,False,False,False,0.0,0.0,False,99.8,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,NaN,retains all,retains all,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,True,2.982778,2.846694,-0.136085


In [19]:
with open('../output/clones_for_LLPS_assay.txt', 'w') as f:
    f.write('\n'.join(sorted(selected_clones)))

In [16]:
df.loc[df['clone_acc_alt'].isin(selected_clones)].sort_values('gene_symbol').to_csv('../output/pairs_selected_for_llps_assay.tsv', sep='\t')

In [17]:
df.loc[df['clone_acc_alt'].isin(selected_clones)
    & (df['gene_symbol'] == 'TP53')]

,gene_symbol,Ensembl_gene_ID,family,is_dimerizing_TF_family,clone_acc_ref,clone_acc_alt,Ensembl_transcript_IDs_ref,Ensembl_transcript_IDs_alt,is_ref_novel_isoform,is_alt_novel_isoform,is_MANE_select_isoform_cloned,n_aa_ref,n_aa_alt,n_exons_ref,n_exons_alt,is_alternative_N_terminal,is_alternative_C_terminal,is_alternative_internal_exon,is_alternative_5_prime_donor,is_alternative_3_prime_acceptor,is_exon_skipping,is_mutually_exclusive_exons,is_intron_retention,dbd_insertion_n_aa,dbd_pct_lost,dbd_affected,aa_seq_pct_id,n_positive_PPI_ref,n_positive_PPI_alt,n_PPI_successfully_tested_in_ref_and_alt,n_positive_PPI_ref_filtered,n_positive_PPI_alt_filtered,n_shared_PPI,PPI_jaccard,dimer_ppi,other_than_dimer_ppi,tf_tf_ppi,tf_cofactor_ppi,tf_signalling_ppi,n_positive_PDI_ref,n_positive_PDI_alt,n_PDI_successfully_tested_in_ref_and_alt,n_positive_PDI_ref_filtered,n_positive_PDI_alt_filtered,n_shared_PDI,PDI_jaccard,at_least_one_isoform_in_gene_abs_activation_gte_2fold,activation_ref,activation_alt,activation_fold_change_log2
337,TP53,ENSG00000141510,p53,True,TP53|1/2|11G10,TP53|2/2|02F03,ENST00000269305|ENST00000445888,ENST00000610292|ENST00000619485|ENST00000620739,False,False,True,393,354,10,8,True,False,False,False,True,False,False,False,0.0,0.0,False,89.8,3.0,0.0,3.0,3.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,2.766992,NaN
